## 빅데이터분석기사 작업형 2

정시 배송 여부 판단
- 데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv
- 데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [2]:
#결측치파악
print(x_train.isna().sum())
print(x_test.isna().sum())

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64
ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64


In [3]:
#불필요한 컬럼 제거
x_test_ID = x_test['ID']
x_train = x_train.drop(columns=['ID'] , axis=1)
x_test = x_test.drop(columns=['ID'] , axis=1)

In [4]:
# 범주형(object), 수치형(number) 변수 파악
obj = x_train.select_dtypes(include = 'object').columns
num = x_train.select_dtypes(include = 'number').columns
print(obj)
print(num)

Index(['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Product_importance', 'Gender'],
      dtype='object')
Index(['Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms'],
      dtype='object')


In [5]:
#범주형 변수(object) 전처리
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['Warehouse_block'] = encoder.fit_transform(x_train['Warehouse_block'])
x_train['Mode_of_Shipment'] = encoder.fit_transform(x_train['Mode_of_Shipment'])
x_train['Customer_care_calls'] = encoder.fit_transform(x_train['Customer_care_calls'])
x_train['Product_importance'] = encoder.fit_transform(x_train['Product_importance'])
x_train['Gender'] = encoder.fit_transform(x_train['Gender'])

x_test['Warehouse_block'] = encoder.fit_transform(x_test['Warehouse_block'])
x_test['Mode_of_Shipment'] = encoder.fit_transform(x_test['Mode_of_Shipment'])
x_test['Customer_care_calls'] = encoder.fit_transform(x_test['Customer_care_calls'])
x_test['Product_importance'] = encoder.fit_transform(x_test['Product_importance'])
x_test['Gender'] = encoder.fit_transform(x_test['Gender'])

In [6]:
# 수치형 변수(number) 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [7]:
# 모델링을 위해 데이터분리(data split)
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train['Reached.on.Time_Y.N'], test_size=0.3, random_state = 1234)

In [8]:
# 모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model.predict_proba(X_TEST)[:,1]

In [9]:
# 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

0.7390591005306592


In [10]:
# 최종제출
y_test_pred = model.predict_proba(x_test)[:,1]
result = pd.DataFrame(y_test_pred)
final = pd.concat([x_test_ID, result], axis = 1).rename(columns = {0:'Reached.on.Time_Y.N'})
final.to_csv('123456.csv', index = False)

In [11]:
# 제출확인
check = pd.read_csv('123456.csv')
print(check)

         ID  Reached.on.Time_Y.N
0      6811                 0.53
1      4320                 0.42
2      5732                 0.36
3      7429                 0.72
4      2191                 1.00
...     ...                  ...
4396   2610                 1.00
4397   3406                 0.32
4398  10395                 0.30
4399   3646                 0.33
4400    573                 1.00

[4401 rows x 2 columns]
